In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
!pip install bert-for-tf2
!pip install sentencepiece
!pip install stormtrooper[setfit]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 656.2 kB/s eta 0:00:000:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30509 sha256=5eab9926520a51108ad9e9aa3de2db804ab2622a49e55a760f20fe54ee10a1dc
  Stored in directory: /root/.cache/pip/wheels/d8/da/50/126d7b8416d9a0e6bf876935c2219a71e72a6529c25e150c56
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19454 sha256=eae42825f51143831a1cf29ff8bcdca256fa7f7a6933a74487ebcd990b322417
  Stored in directory: /root/.cache/pip/wheels/97/a8/d0/f7419404174976a2686bb98b5c30df01cc71445415f32db9e6
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7891 sha256=db7c10aa6ed668a8c2389cc8ef40346a428fea615c5175c62d113add4aa708b4
  Stored in directory: /root/.cache/pip/wheels/69/c8/b3/92666cff9fb312bc3473eaa6b396695b89a7b3e31e9

In [4]:
import re
import math
from bs4 import BeautifulSoup  ## For cleanin the data
import random
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.keras import layers
import bert 

2024-05-05 11:41:45.457193: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 11:41:45.457318: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 11:41:45.621679: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Data Processing

In [5]:
data = pd.read_csv("/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1", engine="python")
data.columns = ["label", "time", "date", "query", "username", "text"]
data.head()

,label,time,date,query,username,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [6]:
data.label.value_counts()

label
4    800000
0    799999
Name: count, dtype: int64

In [7]:
data.drop(["time","date","query","username"], axis= 1, inplace=True)

In [8]:
data.columns=[ 'sentiment','tweet']
data['sentiment'] = data['sentiment'].replace(4,1)
data.head()

,sentiment,tweet
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [9]:
## Instead of working on complete data - create small fraction for test and train

# data_train = data.sample(n=100000, random_state=42)
data_train = data.sample(frac=0.15, random_state=42)
subset2 = data.drop(data_train.index)
data_test = subset2.sample(frac=0.05, random_state=42)

print(f'New sampled train data:', {data_train.shape})
print(f'New sampled train data:', {data_test.shape})

New sampled train data: {(240000, 2)}
New sampled train data: {(68000, 2)}


In [10]:
data_train.sentiment.value_counts()

sentiment
1    120175
0    119825
Name: count, dtype: int64

In [11]:
data_test.sentiment.value_counts()

sentiment
1    34082
0    33918
Name: count, dtype: int64

In [12]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    tweet = re.sub(r"https?://\S+|www\.\S+"," ",tweet) # Removing URLS
    tweet = re.sub(r"@[A-Za-z0-9]+"," ",tweet) # @texts
    tweet = re.sub(r"https?://[A-Za-z0-9]+"," ",tweet) # Removing html tags
    tweet = re.sub(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});"," ",tweet) # Removing html tags
    tweet = re.sub(r"[^\w\s]", " ", tweet) # Removing Punctuation
    tweet = re.sub(r" +", " ", tweet) # Removing repeatition of anything
    tweet = re.sub(r"[^a-zA-Z.!?']", " ", tweet) # Removing non-letter words
    tweet = re.sub(r"\s+", " ", tweet).strip() # Removing whitespace
    return tweet

In [13]:
data_clean = [clean_tweet(tweet) for tweet in data_train.tweet] ## Cleaned Train data

/tmp/ipykernel_34/4071081272.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  tweet = BeautifulSoup(tweet, "lxml").get_text()


In [14]:
data_test_clean = [clean_tweet(tweet) for tweet in data_test.tweet]

/tmp/ipykernel_34/4071081272.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  tweet = BeautifulSoup(tweet, "lxml").get_text()


### Tokenization

In [130]:
FullTokenization = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable = False)

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()

tokenizer = FullTokenization(vocab_file,do_lower_case)

In [131]:
print(tokenizer.tokenize("This is such a weird place to LIVEEEE")) ## Convert to token
print(tokenizer.convert_tokens_to_ids(tokenizer.tokenize("This is such a weird place to LIVEEEE"))) ## Token to IDS
tokenizer.convert_ids_to_tokens([2023, 2003, 2107, 1037, 6881, 2173, 2000, 2444, 4402, 2063]) ## Is to tokens

['this', 'is', 'such', 'a', 'weird', 'place', 'to', 'live', '##ee', '##e']
[2023, 2003, 2107, 1037, 6881, 2173, 2000, 2444, 4402, 2063]


['this', 'is', 'such', 'a', 'weird', 'place', 'to', 'live', '##ee', '##e']

In [132]:
## We will use first sentence  for BERT inputs -  CLS is for classification
def encode_sentence(sent):
    return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [133]:
data_inputs = [encode_sentence(tweet) for tweet in data_clean]

In [134]:
data_inputs[0]

['[CLS]',
 'my',
 'poor',
 'little',
 'dump',
 '##ling',
 'in',
 'holm',
 '##del',
 'vi',
 '##ds',
 'he',
 'was',
 'really',
 'trying',
 'hope',
 'he',
 'don',
 '##t',
 'try',
 'to',
 'hard',
 'tonight',
 'xx',
 '[SEP]']

### Data Creation
We need to create 3 different inputs  for each sentence

In [143]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

## Mask Padding
def get_mask(tokens):
    return np.char.not_equal(tokens,"[PAD]").astype(int)

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if  tok == '[SEP]':
            current_seg_id = 1-current_seg_id
    return seg_ids

In [136]:
data_labels = list(data_train.sentiment)

In [137]:
data_with_len = [[sent,data_labels[i],len(sent)] for i,sent in enumerate(data_inputs)]

In [138]:
data_with_len[0]

[['[CLS]',
  'my',
  'poor',
  'little',
  'dump',
  '##ling',
  'in',
  'holm',
  '##del',
  'vi',
  '##ds',
  'he',
  'was',
  'really',
  'trying',
  'hope',
  'he',
  'don',
  '##t',
  'try',
  'to',
  'hard',
  'tonight',
  'xx',
  '[SEP]'],
 0,
 25]

In [139]:
for i,sent_lab in enumerate(data_with_len):
    if i<3:
        if sent_lab[2]>7:
            print(sent_lab)
    break

[['[CLS]', 'my', 'poor', 'little', 'dump', '##ling', 'in', 'holm', '##del', 'vi', '##ds', 'he', 'was', 'really', 'trying', 'hope', 'he', 'don', '##t', 'try', 'to', 'hard', 'tonight', 'xx', '[SEP]'], 0, 25]


In [144]:
random.shuffle(data_with_len)
data_with_len.sort(key = lambda x: x[2])
# sorted_all = [(sent_lab[0],sent_lab[1]) for sent_lab in data_with_len if sent_lab[2]>7]

sorted_all = [([get_ids(sent_lab[0]), 
                get_mask(sent_lab[0]), 
                get_segments(sent_lab[0])], sent_lab[1])
             for sent_lab in data_with_len if sent_lab[2]>7]

In [145]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all, 
                                              output_types = (tf.int32, tf.int32))

In [146]:
next(iter(all_dataset))

(<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
 array([[ 101, 2434, 1045, 1049, 2204, 2205, 4283,  102],
        [   1,    1,    1,    1,    1,    1,    1,    1],
        [   0,    0,    0,    0,    0,    0,    0,    0]], dtype=int32)>,
 <tf.Tensor: shape=(), dtype=int32, numpy=1>)

In [91]:
BATCH_SIZE = 32
# all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes = ( (None, ), () ))
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes = ( (3,None), () ))

In [93]:
# next(iter(all_batched))

(<tf.Tensor: shape=(32, 8), dtype=int32, numpy=
 array([[27571,  9541, 17139,  9152, 28954, 13900, 10474,  3122],
        [ 7929,  2292,  1055,  2079,  2023,  2057, 19987,  2063],
        [ 4283,  9212,  2072,  8840,  2140,  2200,  3835,  5449],
        [ 2748,  2045,  2003,  1037,  2607,  2011,  2008,  2171],
        [ 1045,  2123,  1056,  2228, 14106, 20627,  2030,  3571],
        [ 2672,  1037,  2210,  4763,  2158,  2007,  1037,  8691],
        [ 2129,  1045,  2777,  2115,  2388,  3084,  2033,  6517],
        [ 1062,  1045,  4478,  1050,  2965,  9119, 23746,  8516],
        [ 2039,  2025,  2061,  3835,  1998,  2220, 20323,  2082],
        [ 4283,  2106,  2008,  2021,  2009,  2134,  1056,  2147],
        [ 2123,  1056,  5390,  2009,  1055,  7929,  3081,  2078],
        [ 4067,  2017,  2004, 17207,  2056,  1045,  2734,  2009],
        [ 3666,  1996,  4564,  2059,  1996,  8692,  3185,  2982],
        [23746,  2015,  2061, 10140,  1998,  2061,  2003,  2016],
        [10047,  5962, 19810

In [94]:
# NB_BATCHES = math.ceil(len(sorted_all)/BATCH_SIZE)
# NB_BATCHES_TEST = NB_BATCHES//10

# all_batched.shuffle(NB_BATCHES)

# ## Test and Train dataset create
# test_dataset = all_batched.take(NB_BATCHES_TEST)
# train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [96]:
# print(test_dataset)
# train_dataset

<_TakeDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


<_SkipDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

## Model Train 
 In BERT, we follow classic method
  - Sentence Tokenization
  - Sentence Embeddings -  Use pretrained model to do this.
  - Build model - any CNN
  

## TRY LLM classifier

In [16]:
data_train.head()

,sentiment,tweet
541200,0,@Nkluvr4eva My poor little dumpling In Holmde...
750,0,I'm off too bed. I gotta wake up hella early t...
766711,0,I havent been able to listen to it yet My spe...
285055,0,now remembers why solving a relatively big equ...
705995,0,"Ate too much, feel sick"


In [19]:
data_train['cleaned_tweet'] = [clean_tweet(tweet) for tweet in data_train.tweet] ## Cleaned Train data
data_train.head()

,sentiment,tweet,cleaned_tweet
541200,0,@Nkluvr4eva My poor little dumpling In Holmde...,My poor little dumpling In Holmdel vids he was...
750,0,I'm off too bed. I gotta wake up hella early t...,I m off too bed I gotta wake up hella early to...
766711,0,I havent been able to listen to it yet My spe...,I havent been able to listen to it yet My spea...
285055,0,now remembers why solving a relatively big equ...,now remembers why solving a relatively big equ...
705995,0,"Ate too much, feel sick",Ate too much feel sick


In [24]:
data_train[data_train.sentiment>0].head()

,sentiment,tweet,cleaned_tweet
1189017,1,on lunch....dj should come eat with me,on lunch dj should come eat with me
1365937,1,@TamaraSchilling Adventure - That's what we al...,Adventure That s what we all need in our life ...
1380170,1,@PerezHilton Zach makes me pee sitting down! A...,Zach makes me pee sitting down And I m a grown...
1233932,1,to sum up my day in one word ......... kackered!,to sum up my day in one word kackered
1495500,1,@k9wkj Great minds think alike,Great minds think alike


In [21]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer, Trainer, TrainingArguments
import torch

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2ForSequenceClassification.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [1]:
from transformers import AutoModelForSequenceClassification

model_name = "llama"
num_labels = 2 # replace with the actual number of labels in your classification task

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

OSError: llama is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [26]:
# Prepare prompt and few-shot examples
prompt = "Sentiment classification: Positive tweets are labeled as 'positive', negative tweets are labeled as 'negative'. Please classify the sentiment of the following tweet:"
positive_example = "I love this movie!"
negative_example = "This movie is terrible!"

In [27]:
# Prepare training data
train_texts = ["positive: " + positive_example, "negative: " + negative_example]
train_labels = [1, 0]  # 1 for positive, 0 for negative